# Sketch Classification

## Load dataset
Loading dataset from local folder. The *load_sketches* returns shuffled data with labels, splitted for training and test.

In [13]:
import load_sketches as ld

In [14]:
sketches, sketch_labels, label_strings = ld.load_sketches("sketches/")

Number of cat sketches:  123202
Number of dog sketches:  152159
Number of flower sketches:  144818
Number of house sketches:  135420
Number of sun sketches:  133781
Total number of sketches:  689380
Total number of labels:  689380


## Set up model

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [17]:
model = Sequential()
#model.add
#model.add

In [18]:
#model.compile()

## Train model

In [ ]:
#model.fit()